# Setup

In [1]:
NEO4J_START_FROM_SCRATCH = True

# NEO4J_WORKDIR = "/var/lib/neo4j"
NEO4J_PORT = 7687
NEO4J_WEBUI_PORT = 7474

NEO4J_INIT_USER = "neo4j"
NEO4J_INIT_PASSWORD = "password"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

### Create session

In [3]:
from neo4j import GraphDatabase

n4j_uri = f"bolt://localhost:{NEO4J_PORT}"
n4j_auth = (NEO4J_INIT_USER, NEO4J_INIT_PASSWORD)

driver = GraphDatabase.driver(n4j_uri, auth=n4j_auth)
session = driver.session()

### Insert nodes

In [4]:
from faker import Faker

fake = Faker()

In [5]:
total_persons = 10

with session.begin_transaction() as transaction:
    transaction.run(
        """
        UNWIND $batch AS user
        MERGE (p:Person {id: user.id})
        SET p.name = user.name, 
            p.email = user.email, 
            p.job = user.job, 
            p.city = user.city,
            p.joined = date(user.joined)
        """,
        batch=[
            {
                "id": i,
                "name": fake.unique.first_name(),
                "email": fake.email(),
                "job": fake.job(),
                "city": fake.city(),
                "joined": fake.date_this_decade().isoformat(),
            }
            for i in range(total_persons)
        ],
    )
    print("Successfully inserted nodes")

Successfully inserted nodes


### Insert relationships

In [6]:
import random

total_relationships = 20

with session.begin_transaction() as transaction:
    user_ids = [
        record["id"] for record in transaction.run("MATCH (p:Person) RETURN p.id AS id")
    ]
    if len(user_ids) < 2:
        print("Not enough users to create friendships.")
    else:
        pairs = set()
        for _ in range(total_relationships):
            ids = tuple(sorted(random.sample(user_ids, 2)))
            pairs.add(ids)
        transaction.run(
            """
        UNWIND $pairs AS pair
        MATCH (a:Person {id: pair[0]})
        MATCH (b:Person {id: pair[1]})
        MERGE (a)-[:knows]->(b)
        MERGE (b)-[:knows]->(a)
        """,
            pairs=[list(pair) for pair in pairs],
        )
        print("Successfully added relationships")

Successfully added relationships


### Delete nodes

In [7]:
with session.begin_transaction() as transaction:

    # transaction.run("MATCH (a:Person)<-[r:knows]-(b:Person) DELETE r")
    transaction.run("MATCH (p:Person) DETACH DELETE p")

    print("Successfully deleted persons")

Successfully deleted persons
